In [ ]:
%pip install webdavclient3 pandas openpyxl

In [2]:
from webdav3.client import Client
import getpass
password = getpass.getpass()

 ········


In [3]:
options = {
       'webdav_hostname': "https://webdav.hidrive.ionos.com/",
       'webdav_login': "xeliba",
       'webdav_password': password,
}

In [4]:
client = Client(options)

In [5]:
def get_files_and_dirs(cur_list):
    flist = [item for item in cur_list if not item["isdir"]]
    dlist = [item for item in cur_list if item["isdir"]]
    return flist, dlist

In [ ]:
big_list = []

def add_files_recursive(dirname):
    global big_list
    cur_list = client.list(dirname, get_info=True)
    flist, dlist = get_files_and_dirs(cur_list)
    big_list += flist
    showname = dirname[:100]
    showname = showname + " " * (101 - len(showname))
    print(f"{showname}{len(flist):10d}{len(big_list):10d}", end="\r")
    for d in dlist:
        add_files_recursive(d["path"])
add_files_recursive("/")

/users/xeliba/BilderBis2015/2010-/2009/Herbst/Krippe/November 2009/4.11.2009/                                20     13822

In [ ]:
import pandas as pd

In [ ]:
data = pd.DataFrame(big_list)

In [ ]:
data["size"].astype(int).sum() / 1024 / 1024 / 1024

In [ ]:
data["etag"] = data.etag.str.strip('"')

In [ ]:
data.to_csv("hidrive_list.csv")

In [ ]:
etags = data.groupby(["etag"]).size()

In [ ]:
etags

In [ ]:
doubles = etags[etags==2].index.tolist()

In [ ]:
data[data.etag.str.startswith("7-57e7dd732dd80")].path.iloc[0]

In [ ]:
for d in doubles:
    for _,s in data[data.etag==d].iterrows():
        print(s.path)

In [ ]:
data.to_csv("hidrive_stuff.csv")

In [ ]:
big_list